# Transforming
It includes the following different transformation
1. Conversion from 1 language to another.
2. Conversion from formal to informal tone.
3. Conversion from JSON to HTML format or vice-versa
4. Check and correct for grammatical mistakes

In [16]:
import json
import openai
import sys
sys.path.append('../')

from IPython.display import display, HTML, Markdown
from redlines import Redlines

# Getting credentials
from Config import openaiConfig
openai.organization = openaiConfig.OPENAI_ORGANISATION
openai.api_key = openaiConfig.OPENAI_API_KEY

In [2]:
def getCompletion(prompt, model='gpt-3.5-turbo'):
    messages = [
    {'role':'user','content':prompt}
    ]
    response = openai.ChatCompletion.create(model = model,
                                            messages = messages,
                                            temperature = 0)
    return response.choices[0].message['content']

## 1. Translation
Converts the input text to different language

In [3]:
userMessages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [5]:
for msg in userMessages:
    prompt = f"Tell me what language this is: ```{msg}```"
    lang = getCompletion(prompt)
    print(f"Original message ({lang}): {msg}")

    prompt = f"""
    Translate the following  text to English \
    and Hindi: ```{msg}```
    """
    response = getCompletion(prompt)
    print(response, "\n")

Original message (This is French.): La performance du système est plus lente que d'habitude.
English: The system performance is slower than usual.
Hindi: सिस्टम कार्यक्षमता आम तौर पर से धीमी है। 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that don't light up.
Hindi: मेरे मॉनिटर में पिक्सेल हैं जो जलने वाले नहीं हैं। 

Original message (This is Italian.): Il mio mouse non funziona
English: My mouse is not working
Hindi: मेरा माउस काम नहीं कर रहा है। 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken.
Hindi: मेरी Ctrl कुंजी टूट गई है। 

Original message (This is Chinese (Simplified).): 我的屏幕在闪烁
English: My screen is flickering.
Hindi: मेरी स्क्रीन फ्लिकरिंग हो रही है। 



## 2. Tone Transformation

In [6]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Shri, check out this awesome GPT3.5 model, it is mind blowing.'
"""
response = getCompletion(prompt)
print(response)

Dear Sir/Madam,

I am writing to bring to your attention the impressive GPT3.5 model that I have recently come across. This model, developed by Shri, is truly remarkable and has the potential to revolutionize the industry.

Thank you for your time and consideration.

Sincerely,

[Your Name]


## 3. Format Conversion

In [9]:
jsonData = { "resturant employees" :[ 
    {"name":"Shri", "email":"shri@gmail.com"},
    {"name":"Tejaswini", "email":"teju@gmail.com"},
    {"name":"Karan", "email":"karan@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {jsonData}
"""
response = getCompletion(prompt)
display(HTML(response))

Name,Email
Shri,shri@gmail.com
Tejaswini,teju@gmail.com
Karan,karan@gmail.com


## 4. Speelcheck/ Grammer check
To signal to the LLM that you want it to proofread your text, you instruct the model to `proofread` or `proofread and correct`.

In [10]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = getCompletion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found.
It's going to be a long day. Does the car need its oil changed?
Their goes my freedom. There going to bring they're suitcases.

Corrected version: 
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
This phrase is to check ChatGPT for spelling ability.


In [14]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = getCompletion(prompt)
print(response)

I got this for my daughter's birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Additionally, it's a bit small for what I paid for it. I think there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [18]:
diff = Redlines(text, response)
display(Markdown(diff.output_markdown))

<span style="color:red;font-weight:700;text-decoration:line-through;">Got </span><span style="color:red;font-weight:700;">I got </span>this for my <span style="color:red;font-weight:700;text-decoration:line-through;">daughter for her </span><span style="color:red;font-weight:700;">daughter's </span>birthday <span style="color:red;font-weight:700;text-decoration:line-through;">cuz </span><span style="color:red;font-weight:700;">because </span>she keeps taking mine from my <span style="color:red;font-weight:700;text-decoration:line-through;">room.  </span><span style="color:red;font-weight:700;">room. </span>Yes, adults also like pandas <span style="color:red;font-weight:700;text-decoration:line-through;">too.  </span><span style="color:red;font-weight:700;">too. </span>She takes it everywhere with her, and it's super soft and <span style="color:red;font-weight:700;text-decoration:line-through;">cute.  One </span><span style="color:red;font-weight:700;">cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. <span style="color:red;font-weight:700;text-decoration:line-through;">It's </span><span style="color:red;font-weight:700;">Additionally, it's </span>a bit small for what I paid for <span style="color:red;font-weight:700;text-decoration:line-through;">it though. </span><span style="color:red;font-weight:700;">it. </span>I think there might be other options that are bigger for the same <span style="color:red;font-weight:700;text-decoration:line-through;">price.  It </span><span style="color:red;font-weight:700;">price. On the positive side, it </span>arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [22]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = getCompletion(prompt)
display(Markdown(response))

Title: A Soft and Cute Panda Plushie for All Ages

As an adult, I can attest that pandas are not just for kids. That's why I got this adorable panda plushie for my daughter's birthday, after she kept taking mine from my room. And let me tell you, it was a hit!

The plushie is super soft and cuddly, making it the perfect companion for my daughter. She takes it everywhere with her, and it has quickly become her favorite toy. However, I did notice that one of the ears is a bit lower than the other, which I don't think was designed to be asymmetrical. But that doesn't take away from its cuteness.

The only downside is that it's a bit small for the price I paid. I think there might be other options that are bigger for the same price. But overall, I'm happy with my purchase.

One thing that surprised me was that it arrived a day earlier than expected. This gave me the chance to play with it myself before giving it to my daughter. And I have to say, I was impressed with the quality and attention to detail.

In conclusion, if you're looking for a soft and cute panda plushie for yourself or a loved one, this is definitely a great option. Just be aware that it might be a bit smaller than expected.